# functools Module

The functools module provides higher-order functions—functions that operate on or return other functions. It is primarily used for functional programming patterns, performance optimization, decorators, and clean API design.

In professional Python development (including data engineering and backend systems), functools is used to:

* Reuse and compose functions
* Improve performance via caching
* Preserve function metadata in decorators
* Create configurable and partial functions
* Enable total ordering and custom comparison logic

In [1]:
import functools as ft

In [2]:
print(ft.__all__)

['update_wrapper', 'wraps', 'WRAPPER_ASSIGNMENTS', 'WRAPPER_UPDATES', 'total_ordering', 'cache', 'cmp_to_key', 'lru_cache', 'reduce', 'partial', 'partialmethod', 'singledispatch', 'singledispatchmethod', 'cached_property', 'Placeholder']


# functools.lru_cache

Purpose: Cache function results to avoid repeated computations.

**Key Points**

* Implements Least Recently Used (LRU) caching
* Common in dynamic programming, API calls, expensive calculations
* maxsize=None → unlimited cache

In [3]:


@ft.lru_cache(maxsize=128)
def fib1(n):
    if n < 2:
        return n
    return fib1(n - 1) + fib1(n - 2)


fib1(10)  # Slow without caching

55

In [4]:
fib1.cache_info()
# fib1.cache_clear()


CacheInfo(hits=8, misses=11, maxsize=128, currsize=11)

# functools.cache

**Purpose**: Simplified, unbounded cache.

Equivalent to @lru_cache(maxsize=None).

In [5]:
@ft.cache
def fib2(n):
    if n < 2:
        return n
    return fib2(n - 1) + fib2(n - 2)


fib2(10)

55

# functools.partial

**Purpose**: Fix some arguments of a function and create a new callable.

**Use Cases:**

* Functional pipelines
* Callbacks
* Configuration-driven code

In [6]:
def add_my_name(name, greeting):
    return f"{greeting}, {name}!"


greet1 = ft.partial(add_my_name, greeting='Hello')
greet1('Shravan')  # "Hello, Alice!"
# greet1.func, greet1.args, greet1.keywords

# list(map(lambda x: 'Hello ' + x, ['Shravan', 'Hanvika'])) #
list(map(greet1, ['Shravan', 'Hanvika']))  # ['Hello, Shravan!', 'Hello, Hanvika!']

['Hello, Shravan!', 'Hello, Hanvika!']

# functools.wraps

**Purpose**: Preserve metadata when writing decorators.

_Without @wraps, add.__name__ and add.__doc__ are lost._

In [7]:


def my_decor(func):
    @ft.wraps(func)
    def wrapper(*args, **kwargs):
        print(f'before calling {func.__name__}')
        x = func(*args, **kwargs)
        print(f'after calling {func.__name__}')
        return x

    return wrapper


@my_decor
def add(a, b):
    """Returns the sum of a and b."""
    return a + b


add(2, 5)

before calling add
after calling add


7

## functools.update_wrapper

**Purpose**: Lower-level version of wraps.

In [8]:


def my_decor(func):
    def wrapper(*args, **kwargs):
        print(f'before calling {func.__name__}')
        x = func(*args, **kwargs)
        print(f'after calling {func.__name__}')
        return x

    ft.update_wrapper(wrapper, func)
    return wrapper


@my_decor
def add(a, b):
    """Returns the sum of a and b."""
    return a + b


add(2, 5)

before calling add
after calling add


7

# functools.reduce

**Purpose**: Apply a binary function cumulatively to an iterable.

**Preferable Alternatives**

* sum() for addition
* math.prod() for multiplication

In [9]:

nums = [1, 2, 3, 4]
result = ft.reduce(lambda x, y: x * y, nums)
result  # ((1*2)*3)*4 = 24

24

In [10]:
names = ['A', 'B', 'C']
result = ft.reduce(lambda x, y: x + ' & ' + y, names)
# takes 1st two elements and applies func, then applies func to result and next element
" & ".join(names)

'A & B & C'

# functools.total_ordering

**Purpose**: Simplify the creation of rich comparison methods.

_**Python automatically provides** : >, <=, >=_

In [11]:

@ft.total_ordering
class Employee:
    def __init__(self, salary):
        self.salary = salary

    def __eq__(self, other):
        return self.salary == other.salary

    def __lt__(self, other):
        return self.salary < other.salary

# functools.singledispatch

**Purpose**: Function overloading based on argument type.

In [12]:

@ft.singledispatch
def handler(value):
    raise NotImplementedError


@handler.register(int)
def _(value):
    return f"Handling an integer: {value}"


@handler.register(str)
def _(value):
    value = list(value)
    return f"Handling a string: {value}"


@handler.register(Exception)
def _(value):
    value = (value.__class__.__name__, str(value))
    return f"Handling an Exception: {value}"


try:
    raise ValueError('its a value error')
except Exception as e:
    print(handler(e))

Handling an Exception: ('ValueError', 'its a value error')


## functools.singledispatchmethod

**Purpose**: Same as singledispatch, but for class methods.

In [13]:
from numbers import Real
from decimal import Decimal


class Handler:
    @ft.singledispatchmethod
    def handle(self, arg):
        raise NotImplementedError

    @handle.register(Real)
    def _(self, arg):
        return Decimal(str(arg)) + 1

    @handle.register(str)
    def _(self, arg):
        return arg + "!"


h = Handler()
h.handle(5)  # 6
h.handle("Hello")  # "Hello!"
h.handle(3.14)  # NotImplementedError
h.handle(5)

Decimal('6')